# Fiber segmentation

In [1]:
import matplotlib.pylab as plt
%matplotlib inline

from pathlib import Path
import numpy as np
import time

from skimage.morphology import skeletonize
from skimage.filters import threshold_otsu
import itk

import sys
import os
sys.path.append(str(Path(os.getcwd()) / '..' / 'src' ))
from itkwidgets import view, compare

import zarr
import xarray as xr

from dask.distributed import Client
from fsspec.implementations.http import HTTPFileSystem
import dask_image.ndmeasure
import dask.array as da
import dask

In [2]:
# Option 1: local cluster

# local_cluster = LocalCluster(n_workers=8, processes=False, memory_limit='4G')
# client = Client(local_cluster)
# client

In [3]:
# Option 2: NERSC Cori Slurm dask-mpi cluster
#
# See SC20_pyHPC/nersc/README.md

scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler.json")
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status"

client = Client(scheduler_file=scheduler_file)
# client

In [4]:
# We can access our API using fsspec's HTTPFileSystem
fs = HTTPFileSystem()
# The http mapper gives us a dict-like interface to the API
store = fs.get_mapper("https://fiber-bed-zarr.netlify.com/rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6.zarr")

In [5]:
ds = xr.open_zarr(store, consolidated=True)
ds

,Array,Chunk
Bytes,14.16 GB,262.14 kB
Shape,"(2160, 2560, 2560)","(64, 64, 64)"
Count,54401 Tasks,54400 Chunks
Type,uint8,numpy.ndarray


In [6]:
fibers = ds.rec20160318_191511_232p3_2cm_cont__4097im_1500ms_ML17keV_6
fibers

,Array,Chunk
Bytes,14.16 GB,262.14 kB
Shape,"(2160, 2560, 2560)","(64, 64, 64)"
Count,54401 Tasks,54400 Chunks
Type,uint8,numpy.ndarray


In [7]:
view(fibers[1000,:,:].values)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [8]:
view(fibers[950:1050,1000:1200,1000:1200].values)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [9]:
image = itk.image_view_from_array(fibers[950:1050,1000:1200,1000:1200].values.astype(np.float64))

In [10]:
sigma = 5.25
hessian = itk.hessian_recursive_gaussian_image_filter(image, sigma=sigma)
vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype('float')].New(hessian)
vesselness_filter.Update()
vesselness = vesselness_filter.GetOutput()
view(vesselness, mode='z', vmax=0.5)

Viewer(geometries=[], gradient_opacity=0.22, mode='z', point_sets=[], rendered_image=<itk.itkImagePython.itkIm…

In [11]:
compare(image, vesselness, gradient_opacity=0.2, shadow=False, mode='z')

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…

In [12]:
thresh = itk.otsu_threshold_image_filter(vesselness)
#compare(image, thresh, interpolate=False, mode='z')

In [13]:
inverted = itk.invert_intensity_image_filter(thresh)
rescaled = itk.rescale_intensity_image_filter(inverted, ttype=(type(inverted), itk.Image[itk.UC, 3]))

In [14]:
view(rescaled)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [15]:
thin = itk.binary_thinning_image_filter(np.asarray(rescaled)[50,:,:])

In [16]:
view(thin)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [17]:
label, nlabels = dask_image.ndmeasure.label(np.asarray(rescaled)[50,:,:])
dask_image.ndmeasure.center_of_mass(label, label, da.unique(label).compute()).compute()

array([[         nan,          nan],
       [  3.42105263, 101.52631579],
       [  5.24731183, 167.94623656],
       [ 11.28282828, 189.62121212],
       [  6.5       ,  30.        ],
       [  8.        , 130.        ],
       [ 10.6       ,  14.2       ],
       [ 15.42168675,  64.27710843],
       [ 12.        ,  88.        ],
       [ 13.09090909, 151.        ],
       [ 15.82352941,  11.55882353],
       [ 16.25      ,  85.        ],
       [ 21.68333333,  38.6       ],
       [ 27.73333333,  52.06666667],
       [ 31.712     , 177.408     ],
       [ 29.9047619 ,   4.07142857],
       [ 28.17647059, 133.70588235],
       [ 30.94444444,  93.5       ],
       [ 33.48076923, 157.55769231],
       [ 36.37362637,  25.06593407],
       [ 41.35064935,  73.88961039],
       [ 37.        , 101.        ],
       [ 40.53846154, 150.92307692],
       [ 39.        , 199.        ],
       [ 42.76666667,  45.7       ],
       [ 46.26086957, 119.63768116],
       [ 46.        ,   9.5       ],
 

In [18]:
view(label_image=label.compute().astype(np.uint16))

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_image=<itk.itk…

In [19]:
fiber_slab = fibers[950:1050,:,:]
fiber_slab

,Array,Chunk
Bytes,655.36 MB,262.14 kB
Shape,"(100, 2560, 2560)","(64, 64, 64)"
Count,59201 Tasks,4800 Chunks
Type,uint8,numpy.ndarray


In [20]:
fiber_slab = fiber_slab.chunk((100,128,128))
fiber_slab

,Array,Chunk
Bytes,655.36 MB,1.64 MB
Shape,"(100, 2560, 2560)","(100, 128, 128)"
Count,59601 Tasks,400 Chunks
Type,uint8,numpy.ndarray


In [21]:
def compute_vesselness(image_chunk):
    import itk
    
    image = itk.image_view_from_array(image_chunk.astype(np.float64))

    sigma = 5.25
    hessian = itk.hessian_recursive_gaussian_image_filter(image, sigma=sigma)
    vesselness_filter = itk.Hessian3DToVesselnessMeasureImageFilter[itk.ctype('float')].New(hessian)
    vesselness_filter.Update()
    vesselness = vesselness_filter.GetOutput()
    return np.asarray(vesselness)

In [22]:
feature = fiber_slab.data.map_overlap(compute_vesselness,
                                depth=32,
                                trim=True,
                                dtype=np.float32)

In [23]:
start = time.time()

feature_result = feature.compute()

elapsed = time.time() - start
# 115.40283918380737 seconds
print(elapsed, 'seconds')

44.990673303604126 seconds


In [24]:
threshold_value = threshold_otsu(np.asarray(vesselness))
threshold_value

0.14946619

In [ ]:
#start = time.time()

#feature_threshold = feature > threshold_value
#label, nlabels = dask_image.ndmeasure.label(feature_threshold[50,:,:])
#label, nlabels = dask.persist(label, nlabels)
#centers = dask_image.ndmeasure.center_of_mass(label, label, da.unique(label).compute()).compute()

#elapsed = time.time() - start

Increasing number of chunks by factor of 20


In [32]:
feature_slice = feature_result[50,:,:]

In [33]:
type(feature_slice)

numpy.ndarray

In [34]:
view(feature_slice, vmax=0.5 )

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF2; pro…

In [38]:
feature_threshold = feature_slice > threshold_value

In [39]:
label, nlabels = dask_image.ndmeasure.label(feature_threshold)

In [40]:
label, nlabels = dask.persist(label, nlabels)

In [42]:
nlabels.compute()

6679

In [43]:
l = label.compute()

In [44]:
l

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [47]:
view(label_image=l.astype(np.uint64))

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_image=<itk.itk…

In [49]:
centers = dask_image.ndmeasure.center_of_mass(l, l, np.unique(l)).compute()

invalid value encountered in double_scalars


In [51]:
centers.shape

(6680, 2)

In [52]:
import pandas as pd

In [56]:
df = pd.DataFrame(centers[1:,:], columns=('X', 'Y'))

In [57]:
df

,X,Y
0,27.583333,1251.833333
1,61.500000,977.000000
2,258.000000,1081.000000
3,266.000000,1383.500000
4,271.500000,1069.000000
...,...,...
6674,2296.725926,1359.651852
6675,2295.000000,1135.000000
6676,2301.574803,1290.874016
6677,2300.766667,1338.133333


In [59]:
df.to_csv('../data/Results-slice1000-fiber_segmentation.csv')

In [60]:
fiber_slice = fiber_slab[50,:,:].values